In [2]:
import pandas as pd


In [3]:
# Load dataset
df = pd.read_csv("Resources/Customer-Churn-Records.csv")

# Display first few rows
df.head()

#print(len(df))

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [4]:
# 1 → Positive/True/Yes
# 0 → Negative/False/No

In [5]:
# Identify missing values
df.isnull().sum() 

#No need to fill missing values with avergaes

RowNumber             0
CustomerId            0
Surname               0
CreditScore           0
Geography             0
Gender                0
Age                   0
Tenure                0
Balance               0
NumOfProducts         0
HasCrCard             0
IsActiveMember        0
EstimatedSalary       0
Exited                0
Complain              0
Satisfaction Score    0
Card Type             0
Point Earned          0
dtype: int64

In [6]:
#check for duplicates
duplicates = df.duplicated().sum()
print("Number of duplicate records:", duplicates)

df.drop_duplicates(inplace=True)

Number of duplicate records: 0


In [7]:
# Convert Binary values to yes or no, 
binary_cols = ["HasCrCard", "IsActiveMember", "Exited", "Complain"]
for col in binary_cols:
    df[col] = df[col].map({0: "No", 1: "Yes"})

df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,Yes,Yes,101348.88,Yes,Yes,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,No,Yes,112542.58,No,Yes,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,Yes,No,113931.57,Yes,Yes,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,No,No,93826.63,No,No,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,Yes,Yes,79084.10,No,No,5,GOLD,425


In [8]:
#Save new cleaned dataset
df.to_csv("Resources/cleaned_customer_churn.csv", index=False)

df_cleaned = pd.read_csv("Resources/cleaned_customer_churn.csv", index_col=False)
df_cleaned = df_cleaned.drop(columns=["RowNumber"])

df_cleaned.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,Yes,Yes,101348.88,Yes,Yes,2,DIAMOND,464
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,No,Yes,112542.58,No,Yes,3,DIAMOND,456
2,15619304,Onio,502,France,Female,42,8,159660.80,3,Yes,No,113931.57,Yes,Yes,3,DIAMOND,377
3,15701354,Boni,699,France,Female,39,1,0.00,2,No,No,93826.63,No,No,5,GOLD,350
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,Yes,Yes,79084.10,No,No,5,GOLD,425


In [9]:
#df_cleaned

In [41]:
print(df.dtypes)

#df_cleaned["HasCrCard"] = df_cleaned["HasCrCard"].astype(str)

for _, row in df_cleaned.iterrows():
    print(tuple(row))  # Debugging line
    break  # Print only one row


RowNumber               int64
CustomerId              int64
Surname                object
CreditScore             int64
Geography              object
Gender                 object
Age                     int64
Tenure                  int64
Balance               float64
NumOfProducts           int64
HasCrCard              object
IsActiveMember         object
EstimatedSalary       float64
Exited                 object
Complain               object
Satisfaction Score      int64
Card Type              object
Point Earned            int64
dtype: object
(15634602, 'Hargrave', 619, 'France', 'Female', 42, 2, 0.0, 1, 'Yes', 'Yes', 101348.88, 'Yes', 'Yes', 2, 'DIAMOND', 464)


In [39]:
#load data into MySQL
#run this section of code after create database and table in MySQL
import pymysql
from dotenv import load_dotenv #use to get .env file
import os

# Load environment variables
load_dotenv()

# Get credentials from .env file
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")

# Connect to MySQL
conn = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

cursor = conn.cursor()

for _, row in df_cleaned.iterrows():
    sql = """
    INSERT INTO Customers (CustomerId, Surname, CreditScore, Geography, Gender, Age, Tenure, Balance,
                           NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary, Exited, Complain,
                           SatisfactionScore, CardType, PointEarned)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = tuple(row.astype(str))  # Convert all values to strings to avoid formatting issues
    cursor.execute(sql, values)

conn.commit()
conn.close()



In [40]:
## Connect data to MySQL
# Check is table is connected
import pymysql
from dotenv import load_dotenv
import os


# Load environment variables
load_dotenv()

# Get credentials from .env file
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")

# Connect to MySQL
conn = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

cursor = conn.cursor()

# Test connection
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("Tables in database:", tables)

# Close connection
conn.close()


Tables in database: (('customers',),)


In [37]:
#pip install pymysql
#pip install python-dotenv


In [4]:
#conn.close()


In [3]:
import pandas as pd


df_cleaned = pd.read_csv("Resources/cleaned_customer_churn.csv", index_col=False)
df_cleaned = df_cleaned.drop(columns=["RowNumber"])

df_cleaned.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,Yes,Yes,101348.88,Yes,Yes,2,DIAMOND,464
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,No,Yes,112542.58,No,Yes,3,DIAMOND,456
2,15619304,Onio,502,France,Female,42,8,159660.80,3,Yes,No,113931.57,Yes,Yes,3,DIAMOND,377
3,15701354,Boni,699,France,Female,39,1,0.00,2,No,No,93826.63,No,No,5,GOLD,350
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,Yes,Yes,79084.10,No,No,5,GOLD,425


In [5]:
df_cleaned.describe()

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Satisfaction Score,Point Earned
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,100090.239881,3.013800,606.515100
std,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,57510.492818,1.405919,225.924839
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,11.580000,1.000000,119.000000
25%,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,51002.110000,2.000000,410.000000
50%,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,100193.915000,3.000000,605.000000
75%,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,149388.247500,4.000000,801.000000
max,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,199992.480000,5.000000,1000.000000
